<a href="https://colab.research.google.com/github/theobringino/cybersecurity/blob/main/ds-ai-ml-projects/projects/llm_fine_tuning/llm_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine Tuning

Note: Select T4 GPU as the runtime here to avoid errors.

## Setup and Loading

#### Environment Setup

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
!pip install -q -U transformers datasets peft bitsandbytes accelerate trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 138.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.0/557.0 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 20.4 MB/s eta 0:00:00


In [30]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTConfig, SFTTrainer

In [33]:
pip install --upgrade trl

#### Model Loading

In [4]:
# Quantization
model_id = "unsloth/llama-3-8b-bnb-4bit"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

In [5]:
# Model and Tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:239: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [6]:
# Low-rankk Adapation
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [7]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, config)

## Data Preparation

Load alpaca dataset, first 1000 rows only

In [9]:
dataset = load_dataset("yahma/alpaca-cleaned", split="train[:1000]")

In [37]:
# Format the data into a prompt that the model can understand
def formatting_prompts_func(example):
    text = (
        f"Instruction:\n{example['instruction']}\n\n"
        f"Response:\n{example['output']}"
    )
    return text

## Model Training

In [34]:
training_args = TrainingArguments(
    output_dir="./outputs",
    max_steps=100,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    optim="paged_adamw_8bit",
    report_to="none"
)

In [38]:
tokenizer.model_max_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    formatting_func=formatting_prompts_func,
)

Applying formatting function to train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

## Testing

Query the model "Explain why the sky is blue like a five-year-old."

In [39]:
instruction = input()
inputs = tokenizer(f"Instruction:\n{instruction}\n\nResponse:\n", return_tensors="pt").to("cuda")


Explain why the sky is blue like a five-year-old.


In [40]:
outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Instruction:
Explain why the sky is blue like a five-year-old.

Response:
The sky is blue because the sun is white. The sun is a very hot, bright ball of gas that is at the center of our solar system. The sun is the main source of energy for our planet. The sun gives off light and heat, which we can see and feel. The light from the sun is white. But when the light from the sun hits the air in the atmosphere, it scatters. The light scatters in all directions. The blue light is scattered more than the
